In [5]:
import pandas as pd
from google.cloud import bigquery
import os
from google.oauth2 import service_account
import re
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [7]:
# Global inputs
# Standort corrections
corrections_standort = [
    {"duisburg": ["duisubrg", "duis", "duisburgburg"]},
    {"hamburg": ["hamburg harburg", "hamburg norderstedt", "hh hamburg", "hh harburg"]},
    {"zurich": ["zürich", "dfp zurich"]}
]

# Special characters
special_chars = ["(", ")", "/", "-", ",", ".", "+"]

# Umlauts
umlauts = {"ä": "ae", "ö": "oe", "ü": "ue", "ß": "ss"}

# Months to take into account
mon_incl = "jan|mai|aug"

# Skill categories for the widgets under the bar chart
skill_cats_widget = ["linear_sprint_30", "agility_mit_ball", "agility_ohne_ball", "jonglieren_8er", "dribbling_8er", "passspiel_ballkontrolle"]

# is_use_gsheet variable
is_use_gsheet = True

# Google Sheet ID
google_sheet_id = "1DALKfwo40QMFRjLLySAvLDjc5yb9sCK9c5eH5r19h_k"

In [3]:
# Schwerpunkte
# 1. Technik - Ballkontrolle
schwerpunkte_1 = {
    # Tonis Excel sheet
    "1. Technik - Ball Kontrolle": [
        "Ballgewöhnung - Innenseite rechts",
        "Ballgewöhnung - Vollspann rechts",
        "Ballgewöhnung - Oberschenkel rechts + Vollspann links",
        "Ballgewöhnung - Brust + Innenseite rechts",
        "Liften - Vollspann rechts",
        "Droppkick - Vollspann rechts",
        "Ballgewöhnung - Innenseite links",
        "Ballgewöhnung - Vollspann links",
        "Ballgewöhnung - Oberschenkel links + Vollspann rechts",
        "Ballgewöhnung - Brust + Innenseite links",
        "Liften - Vollspann links",
        "Droppkick - Vollspann links",
        "Jonglieren mit Rückdrall links und rechts",
        "Jonglieren mit Tennisball links und rechts",
        "Jonglieren - großer Kreis (Vollspann r. - Oberschenkel r. - Schulter r. - Kopf - Schulter l. - Oberschenkel l. - Vollspann l.",
        "Jonglieren mit Rückdrall links und rechts mit dem Mini-Ball",
        "Jonglieren - kleiner Kreis (Vollspann r. - Oberschenkel r. - Kopf - Oberschenkel l. - Vollspann l.",
        "Coerver Level-1",
        "Coerver Level-2",
        "Coerver Level-3",
        # Data existing in datasets
        "jonglieren_mit_rueckdrall",
        "jonglieren_mit_rueckdrall_links_und_rechts"
    ]
}

###-----------------###-----------------###

# 2. Technik - Ballführung, Dribbling, Finte, 1 gegen 1 offensiv, 1 gegen 1 defensiv
schwerpunkte_2 = {
    # Tonis Excel sheet
    "2. Technik - Ballführung, Dribbling, Finte, 1_gegen_1_offensiv, 1_gegen_1_defensiv": [
        "Ballführung/Dribbling rechts",
        "Finte - Körperfinte rechts",
        "Finte - Doppelte Körperfinte rechts",
        "Finte - Übersteiger rechts",
        "Finte - Doppelter Übersteiger Rechts",
        "Finte - Kappen Innenseite rechts",
        "Finte - Kappen Außenseite rechts",
        "Finte - Hinten des rechten Standbeines",
        "Ballführung/Dribbling links",
        "Finte - Körperfinte links",
        "Finte - Doppelte Körperfinte links",
        "Finte - Übersteiger links",
        "Finte - Doppelter Übersteiger links",
        "Finte - Kappen Innenseite links",
        "Finte - Kappen Außenseite links",
        "Finte - Hinten des linken Standbeines",
        "1 gegen 1 - Offensivverhalten",
        "1 gegen 1 - Defensivverhalten",
        "Spielintelligenz / Kreativität",
        # Data existing in datasets
        "1_gegen_1_defensiv",
        "1_gegen_1_offensiv",
        "ballhaendling_innenseite_vollspann_oberschenkel_brust_liftenvolley",
        "dribbling",
        "tempodribbling_lire",
        "finten",
        "koerperfinte_lire",
        "enge_ballfuehrung_lr",
        "ballfuehrung"
    ]
}

###-----------------###-----------------###

# 3. Technik - Passpiel, Ballannahme, Ballmitnahme, Erster Kontakt
schwerpunkte_3 = {
    # Tonis Excel sheet
    "3. Technik - Passpiel, Ballannahme, Ballmitnahme, Erster_Kontakt": [
        "Passspiel - Innenseite Rechts",
        "Gehobener Ball rechts",
        "Ballannahme aus der Luft - rechte Innenseite",
        "Ballannahme aus der Luft - rechte Außenseite",
        "Ballannahme aus der Luft - rechter Vollspann",
        "Ballannahme aus der Luft - rechter Oberschenkel",
        "Ballannahme aus der Luft - Brust",
        "Ballmitnahme aus der Luft Dropkick -  90 Grad - rechte Innenseite",
        "Ballmitnahme aus der Luft  Dropkick - 180 Grad - rechte Innenseite",
        "Ballmitnahme aus der Luft Dropkick -  90 Grad - rechte Außenseite",
        "Ballmitnahme aus der Luft  Dropkick - 180 Grad - rechte Außenseite",
        "Ballmitnahme - Erster Kontakt - nach vorne - rechte Innenseite",
        "Ballmitnahme - Erster Kontakt - 90 Grad - rechte Innenseite",
        "Ballmitnahme - Erster Kontakt mit offener Stellung - ca. 90-120 Grad - rechte Innenseite",
        "Ballmitnahme - Erster Kontakt - 90 Grad - rechte Außenseite",
        "Ballmitnahme - Erster Kontakt - 180 Grad - rechte Außenseite",
        "Passspiel - Innenseite links",
        "Gehobener Ball links",
        "Ballannahme aus der Luft - linke Innenseite",
        "Ballannahme aus der Luft - linke Außenseite",
        "Ballannahme aus der Luft - linker Vollspann",
        "Ballannahme aus der Luft - linker Oberschenkel",
        "Ballmitnahme aus der Luft Dropkick - 90 Grad - linke Innenseite",
        "Ballmitnahme aus der Luft Dropkick - 180 Grad - linke Innenseite",
        "Ballmitnahme aus der Luft Dropkick -  90 Grad - linke Außenseite",
        "Ballmitnahme aus der Luft  Dropkick - 180 Grad - linke Außenseite",
        "Ballmitnahme - Erster Kontakt - nach vorne - linke Innenseite",
        "Ballmitnahme - Erster Kontakt - 90 Grad - linke Innenseite",
        "Ballmitnahme - Erster Kontakt mit offener Stellung - ca. 90-120 Grad - linke Innenseite",
        "Ballmitnahme - Erster Kontakt - 90 Grad - linke Außenseite",
        "Ballmitnahme - Erster Kontakt - 180 Grad - linke Außenseite",
        # Data existing in datasets
        "ballmitnahme_erster_kontakt_180_grad_linke_aussenseite",
        "ballmitnahme_erster_kontakt_180_grad_rechte_aussenseite",
        "ballmitnahme_erster_kontakt_90_grad_linke_aussenseite",
        "ballmitnahme_erster_kontakt_90_grad_linke_innenseite",
        "ballmitnahme_erster_kontakt_90_grad_rechte_aussenseite",
        "ballmitnahme_erster_kontakt_90_grad_rechte_innenseite",
        "ballmitnahme_erster_kontakt_mit_offener_stellung_ca_90120_grad_linke_innenseite",
        "ballmitnahme_erster_kontakt_mit_offener_stellung_ca_90120_grad_rechte_innenseite",
        "ballmitnahme_erster_kontakt_nach_vorne_linke_innenseite",
        "ballmitnahme_erster_kontakt_nach_vorne_rechte_innenseite",
        "passspiel_innenseite_rechts",
        "passspiel_innenseite_links"
    ]
}

###-----------------###-----------------###

# 4. Technik - Torabschluss
schwerpunkte_4 = {
    # Tonis Excel sheet
    "4. Technik - Torabschluss": [
        "Torschuss - rechte Innenseite",
        "Torschuss - rechter Vollspann",
        "Torschuss - Schlenzen Innenseite + Vollspann rechts",
        "Torschuss - Schlenzen Außenseite + Vollspann rechts",
        "Kopfballspiel",
        "Torschuss (Innenseite Links)",
        "Torschuss (Vollspann Links)",
        "Torschuss (Schlenzen Innenseite + Vollspann Links)",
        "Torschuss (Schlenzen Außenseite + Vollspann Links)",
        # Data existing in datasets
        "torschuss",
        "torschuss_innenseite_lire",
        "torschuss_vollspann_lire"
    ]
}

###-----------------###-----------------###

# 5. Athletische Fähigkeiten
schwerpunkte_5 = {
    # Tonis Excel sheet
    "5. Athletische Fähigkeiten": [
        "Handlungsschnelligkeit",
        "Bewegungsschnelligkeit ohne Ball",
        "Reaktionsschnelligkeit (0 Meter)",
        "Antrittsschnelligkeit (3-4 Meter)",
        "Beschleunigungsvermögen (10-30 Meter)",
        "Schnelligkeitsausdauer (bis 80 Meter)",
        "Aktionsschnelligkeit mit Ball",
        "Schnellkraft (Sprungkraft)",
        "Rumpfkraft (Planks)",
        "Fussbalspezifische aerobe Ausdauer",
        "Fussbalspezifische anaerobe Ausdauer",
        # Data existing in datasets
        "aktionsschnelligkeit",
        "aktionsschnelligkeit_bewegung_mit_ball",
        "antrittsschnelligkeit",
        "grundschnelligkeit",
        "laufbewegungsgeschicklichkeit",
        "uebersteiger_lire"
    ]
}

###-----------------###-----------------###

# 6. Kognitive Schnelligkeit
schwerpunkte_6 = {
    # Tonis Excel sheet
    "6. Kognitive Schnelligkeit": [
        # Tonis Excel sheet
        "Entscheidungsschnelligkeit",
        "Antizipationsschnelligkeit",
        "Wahrnehmungsschnelligkeit",
        "Kreativität",
        # Data existing in datasets
        "spielintelligenz_kreativitaet"
    ]
}

###-----------------###-----------------###

# 7. Mentale Fähigkeiten
schwerpunkte_7 = {
    "7. Mentale Fähigkeiten": [
        # Tonis Excel sheet
        "Durchsetzungsvermögen",
        "Lernbereitschaft",
        "Teamfähigkeit",
        # Data existing in datasets
        "disziplin",
        "durchsetzungsvermoegen",
        "durchsetzungsvermoegen_willenstaerke",
        "konzentrationsvermoegen",
        "kritikfaehigkeit",
        "selbstvertrauen",
        "willensstaerke"
    ]
}

In [4]:
# Create a dataframe containing all schwerpunkte
df_schwerpuntke = []
for i in range (1, 8):
    # Create a dataframe for each schwerpunkt
    df_iter = vars()["schwerpunkte_" + str(i)]
    df_schwerpuntke_iter = pd.DataFrame(df_iter)

    # Add a new column "schwerpunkt_id" and fill it with the value of the column name
    df_schwerpuntke_iter["schwerpunkt_id"] = df_schwerpuntke_iter.columns[0]
    
    # Rename the first column to "schwerpunkt"
    df_schwerpuntke_iter = df_schwerpuntke_iter.rename(columns={df_schwerpuntke_iter.columns[0]: "schwerpunkt"})

    # Append the dataframe to the list of dataframes
    df_schwerpuntke.append(df_schwerpuntke_iter)

# Concatenate all dataframes in the list to one dataframe
df_schwerpuntke = pd.concat(df_schwerpuntke)

# Add a new column identifying whether the Schwerpunkt comes from Tonis Excel sheet or from the datasets
# If all characters are small case, then it comes from the datasets. Otherwise, it comes from Tonis Excel sheet
df_schwerpuntke["source"] = df_schwerpuntke["schwerpunkt"].str.islower()
df_schwerpuntke["source"] = df_schwerpuntke["source"].replace({True: "datasets", False: "toni"})

# Clean the data
# Replace all spaces in column names with underscores and remove all special characters
for char in special_chars:
    df_schwerpuntke["schwerpunkt"] = df_schwerpuntke["schwerpunkt"].str.replace(char, "")

# Replace the umlauts with the corresponding letters
for umlaut in umlauts:
    df_schwerpuntke["schwerpunkt"] = df_schwerpuntke["schwerpunkt"].str.replace(umlaut, umlauts[umlaut])

# Strip extra spaces at the end of the column names, the double spaces with single spaces, all spaces with underscores, and change all letters to lowercase
df_schwerpuntke["schwerpunkt"] = df_schwerpuntke["schwerpunkt"].str.strip().str.replace("  ", " ").str.replace(" ", "_").str.lower()

# Add a prefix "T{schwerpunkt_id}_" to the first character of the schwerpunkt_id column
df_schwerpuntke["schwerpunkt"] = "T" + df_schwerpuntke["schwerpunkt_id"].str[0] + "_" + df_schwerpuntke["schwerpunkt"]

# Sort the values of df_schwerpuntke by the schwerpunkt then source columns
df_schwerpuntke = df_schwerpuntke.sort_values(["schwerpunkt", "source"])

# Eliminate the duplicates by subsetting on the schwerpunkt column
df_schwerpuntke = df_schwerpuntke.drop_duplicates(subset=["schwerpunkt"], keep="first")

# Create a new column "schwerpunkt_ohne_prefix" and fill it with the schwerpunkt column without the prefix
df_schwerpuntke["schwerpunkt_ohne_prefix"] = df_schwerpuntke["schwerpunkt"].str[3:]

# Display the schwerpunkte data frame
display(df_schwerpuntke)

# Create a list of all schwerpunkte
print("List of all schwerpunkte:")
df_schwerpuntke["schwerpunkt"].sort_values().unique()

,schwerpunkt,schwerpunkt_id,source,schwerpunkt_ohne_prefix
9,T1_ballgewoehnung_brust_innenseite_links,1. Technik - Ball Kontrolle,toni,ballgewoehnung_brust_innenseite_links
3,T1_ballgewoehnung_brust_innenseite_rechts,1. Technik - Ball Kontrolle,toni,ballgewoehnung_brust_innenseite_rechts
6,T1_ballgewoehnung_innenseite_links,1. Technik - Ball Kontrolle,toni,ballgewoehnung_innenseite_links
0,T1_ballgewoehnung_innenseite_rechts,1. Technik - Ball Kontrolle,toni,ballgewoehnung_innenseite_rechts
8,T1_ballgewoehnung_oberschenkel_links_vollspann...,1. Technik - Ball Kontrolle,toni,ballgewoehnung_oberschenkel_links_vollspann_re...
...,...,...,...,...
7,T7_kritikfaehigkeit,7. Mentale Fähigkeiten,datasets,kritikfaehigkeit
1,T7_lernbereitschaft,7. Mentale Fähigkeiten,toni,lernbereitschaft
8,T7_selbstvertrauen,7. Mentale Fähigkeiten,datasets,selbstvertrauen
2,T7_teamfaehigkeit,7. Mentale Fähigkeiten,toni,teamfaehigkeit


List of all schwerpunkte:


array(['T1_ballgewoehnung_brust_innenseite_links',
       'T1_ballgewoehnung_brust_innenseite_rechts',
       'T1_ballgewoehnung_innenseite_links',
       'T1_ballgewoehnung_innenseite_rechts',
       'T1_ballgewoehnung_oberschenkel_links_vollspann_rechts',
       'T1_ballgewoehnung_oberschenkel_rechts_vollspann_links',
       'T1_ballgewoehnung_vollspann_links',
       'T1_ballgewoehnung_vollspann_rechts', 'T1_coerver_level1',
       'T1_coerver_level2', 'T1_coerver_level3',
       'T1_droppkick_vollspann_links', 'T1_droppkick_vollspann_rechts',
       'T1_jonglieren_grosser_kreis_vollspann_r_oberschenkel_r_schulter_r_kopf_schulter_l_oberschenkel_l_vollspann_l',
       'T1_jonglieren_kleiner_kreis_vollspann_r_oberschenkel_r_kopf_oberschenkel_l_vollspann_l',
       'T1_jonglieren_mit_rueckdrall',
       'T1_jonglieren_mit_rueckdrall_links_und_rechts',
       'T1_jonglieren_mit_rueckdrall_links_und_rechts_mit_dem_miniball',
       'T1_jonglieren_mit_tennisball_links_und_rechts',
       

In [37]:
# Define a function that reads all tabs of a Google Sheet
def google_sheet_reader(sheet_key):
    # Replace 'your_service_account.json' with the filename of your service account key.
    SERVICE_ACCOUNT_FILE = os.path.expanduser("~") + "/service_account_key.json"

    # Authenticate with Google Sheets API using the service account credentials
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet
    spreadsheet = client.open_by_key(sheet_key)

    # Loop over all the worksheets and extract the data
    df_list = []
    for wk in spreadsheet.worksheets():
        # Get all values from the worksheet
        df = pd.DataFrame(wk.get_all_records(empty2zero=False, default_blank=None))

        # Add a new column called "jahr" and fill it with the year of the worksheet
        df["jahr"] = "20" + str(re.findall(pattern=r"(?<=_).*", string=wk.title)[0])

        # Append to the list of dataframes
        df_list.append(df)
    
    # Concatenate all dataframes in the list to one dataframe
    df = pd.concat(df_list)

    # Return df
    return df

In [38]:
if is_use_gsheet == False:
    # If the path does not contain csv_dateien, then the current working directory will be add it to the path
    if "csv_dateien" not in os.getcwd():
        os.chdir(os.getcwd() + "/csv_dateien")
        
    # List all CSV files in the current directory starting with the word "spielerbeurteilung"
    csv_files = [f for f in os.listdir('.') if os.path.isfile(f) and f.startswith("spielerbeurteilung") and f.endswith(".csv") and re.findall(mon_incl, f) != []]
    print(csv_files)
    # Import the dataset
    df_list = []
    for filename in csv_files:
        df_iter = pd.read_csv(filename, encoding="unicode_escape", sep=";")
        # Extract the year from the file name and create a new column called year showing the year of the dataset
        df_iter["jahr"] = "20" + str(filename.split("_")[2].split(".")[0])
        df_list.append(df_iter)

    df = pd.concat(df_list)
else:
    df = google_sheet_reader(google_sheet_id)

# Display the dataframe
df

/var/folders/_q/rc6k40v54ms2vz5jw4m18vf00000gn/T/ipykernel_92938/84910364.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(df_list)


,monat,name_spieler,trainer,standort,position,groesse,nationalitaet,alter,paket,vertrag,...,ballmitnahme_erster_kontakt_nach_vorne_linke_innenseite,ballmitnahme_erster_kontakt_90_grad_rechte_innenseite,ballmitnahme_erster_kontakt_90_grad_linke_innenseite,ballmitnahme_erster_kontakt_mit_offener_stellung_ca._90-120_grad_rechte_innenseite,ballmitnahme_erster_kontakt_mit_offener_stellung_ca._90-120_grad_linke_innenseite,ballmitnahme_erster_kontakt_90_grad_rechte_aussenseite,ballmitnahme_erster_kontakt_90_grad_linke_aussenseite,ballmitnahme_erster_kontakt_180_grad_rechte_aussenseite,ballmitnahme_erster_kontakt_180_grad_linke_aussenseite,passspiel_innenseite_rechts
0,Januar,tim_bangeter,henry,bern,torwart,100,deutschland,07.10.2009,excellence_plus_499,30.11.2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Januar,leandro_campititiello,henry,bern,torwart,100,deutschland,03.05.2005,excellence_plus_499,30.11.2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Januar,levin_vrankic,henry,bern,torwart,100,deutschland,15.03.2009,excellence_plus_499,30.11.2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Januar,leon_salihu,henry,bern,torwart,100,deutschland,31.10.2013,excellence_399_1-Training,30.11.2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Januar,berg_luca,sergio,duisburg,torwart,100,deutschland,18.04.2010,basic_199,30.03.2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Dezember,leonardo_friz_(miu),sergio,duisburg,torwart,100,deutschland,24.11.2009,excellence_399,30.08.2024,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
92,Dezember,max_raabe,sergio,duisburg,torwart,100,deutschland,30.06.2009,excellence_399,30.10.2024,...,6.0,6.0,7.0,6.0,6.0,6.0,6.0,7.0,6.0,6.0
93,Dezember,james_rusch,sergio,duisburg,torwart,100,deutschland,19.08.2010,excellence_399,30.10.2024,...,6.0,6.0,7.0,7.0,6.0,3.0,7.0,7.0,7.0,6.0
94,Dezember,ben_steinbring,sergio,duisburg,torwart,100,deutschland,22.10.2014,excellence_399,31.03.2024,...,6.0,6.0,6.0,7.0,6.0,6.0,6.0,6.0,6.0,7.0


In [39]:
# Make the names of all columns lowercase
df.columns = map(str.lower, df.columns)

# Replace all spaces in column names with underscores and remove all special characters
for char in special_chars:
    df.columns = df.columns.str.replace(char, "")

# Replace the umlauts with the corresponding letters
umlauts = {"ä": "ae", "ö": "oe", "ü": "ue", "ß": "ss"}
for umlaut in umlauts:
    df.columns = df.columns.str.replace(umlaut, umlauts[umlaut])

# Strip extra spaces at the end of the column names
df.columns = df.columns.str.strip()

# Replace the double spaces with single spaces
df.columns = df.columns.str.replace("  ", " ")

# Replace all spaces with underscores
df.columns = df.columns.str.replace(" ", "_")

# Replace all the double underscores with single underscores
df.columns = df.columns.str.replace("__", "_")

df.columns

Index(['monat', 'name_spieler', 'trainer', 'standort', 'position', 'groesse',
       'nationalitaet', 'alter', 'paket', 'vertrag', 'linear_sprint_30',
       'agility_mit_ball', 'agility_ohne_ball', 'jonglieren_8er',
       'dribbling_8er', 'passspiel_ballkontrolle', 'bemerkungen_und_hinweise',
       'jahr', 'mentaltraining', 'ballfuehrung', 'finten', 'dribbling',
       '1_gegen_1_offensiv', '1_gegen_1_defensiv', 'torschuss',
       'antrittsschnelligkeit', 'grundschnelligkeit',
       'lauf_bewegungsgeschicklichkeit', 'aktionsschnelligkeit',
       'spielintelligenz_kreativitaet', 'durchsetzungsvermoegen',
       'willensstaerke', 'konzentrationsvermoegen', 'selbstvertrauen',
       'kritikfaehigkeit', 'jonglieren_mit_rueckdrall',
       'ballhandling_innenseitevollspannoberschenkelbrustliftenvolley',
       'enge_ballfuehrung_lire', 'tempodribbling_lire', 'koerperfinte_lire',
       'uebersteiger_lire', 'torschuss_innenseite_lire',
       'torschuss_vollspann_lire', '1_gegen_1_offe

In [40]:
# Eliminate the NULL months
df = df[df["monat"].notnull()]

# Strip the extra spaces at the end of the month names
df["monat"] = df["monat"].str.lower().str.strip()

# Take the first three letters of any month
df["monat"] = df["monat"].str[:3]
df["monat"].unique()

array(['jan', 'mai', 'aug', 'okt', 'nov', 'dez'], dtype=object)

In [41]:
# Drop columns that are called bemerkungen_und_hinweise
df = df.drop(columns=[col for col in df.columns if "bemerkungen_und_hinweise" in col])

# Fill all "-" values with None
df = df.replace("-", None)

# Strip the extra spaces at the end of the trainer and standort names
for col in ["trainer", "standort"]:
    df[col] = df[col].str.lower().str.strip()

# Change any occurences of [duis, duisburg, or duisubrg] to duisburg
for corr in corrections_standort:
    for item in list(corr.values())[0]:
        df["standort"] = df["standort"].str.replace(item, list(corr.keys())[0])

# Add the column "monat_jahr", which is the concatenation of the columns "jahr" and "monat"
df["monat_jahr"] = df["monat"] + "_" + df["jahr"]

# Add "monat_num" column, which is the number of the month
df["monat_num"] = df["monat"].map(
    {
        "jan": 1,
        "feb": 2,
        "mar": 3,
        "apr": 4,
        "mai": 5,
        "jun": 6,
        "jul": 7,
        "aug": 8,
        "sep": 9,
        "okt": 10,
        "nov": 11,
        "dez": 12,
    }
)

# Melt the dataset so that all columns after standort excluding bemerkungen_und_hinweise are in one column
df_melted = pd.melt(
    df,
    id_vars=[
        "monat_jahr", "jahr", "monat", "monat_num", "name_spieler", "trainer", "standort",
        "position", "groesse", "nationalitaet", "alter", "paket", "vertrag"
    ],
    var_name="kriterium",
    value_name="bewertung",
)

# Change the data type of the bewertung column to float
df_melted["bewertung"] = df_melted["bewertung"].astype(float)
df_melted

,monat_jahr,jahr,monat,monat_num,name_spieler,trainer,standort,position,groesse,nationalitaet,alter,paket,vertrag,kriterium,bewertung
0,jan_2023,2023,jan,1,tim_bangeter,henry,bern,torwart,100,deutschland,07.10.2009,excellence_plus_499,30.11.2024,linear_sprint_30,NaN
1,jan_2023,2023,jan,1,leandro_campititiello,henry,bern,torwart,100,deutschland,03.05.2005,excellence_plus_499,30.11.2024,linear_sprint_30,NaN
2,jan_2023,2023,jan,1,levin_vrankic,henry,bern,torwart,100,deutschland,15.03.2009,excellence_plus_499,30.11.2024,linear_sprint_30,NaN
3,jan_2023,2023,jan,1,leon_salihu,henry,bern,torwart,100,deutschland,31.10.2013,excellence_399_1-Training,30.11.2024,linear_sprint_30,NaN
4,jan_2023,2023,jan,1,berg_luca,sergio,duisburg,torwart,100,deutschland,18.04.2010,basic_199,30.03.2024,linear_sprint_30,5.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39515,dez_2023,2023,dez,12,leonardo_friz_(miu),sergio,duisburg,torwart,100,deutschland,24.11.2009,excellence_399,30.08.2024,passspiel_innenseite_rechts,7.00
39516,dez_2023,2023,dez,12,max_raabe,sergio,duisburg,torwart,100,deutschland,30.06.2009,excellence_399,30.10.2024,passspiel_innenseite_rechts,6.00
39517,dez_2023,2023,dez,12,james_rusch,sergio,duisburg,torwart,100,deutschland,19.08.2010,excellence_399,30.10.2024,passspiel_innenseite_rechts,6.00
39518,dez_2023,2023,dez,12,ben_steinbring,sergio,duisburg,torwart,100,deutschland,22.10.2014,excellence_399,31.03.2024,passspiel_innenseite_rechts,7.00


In [44]:
# Upload the dataset to BigQuery
key_path_home_dir = os.path.expanduser("~") + "/bq_credentials.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path_home_dir, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Now, instantiate the client and upload the table to BigQuery
bq_client = bigquery.Client(project="web-scraping-371310", credentials=credentials)

# Create the job configuration of df_booked_appts
job_config_fussball_akademie = bigquery.LoadJobConfig(
    schema = [
        bigquery.SchemaField("monat_jahr", "STRING"),
        bigquery.SchemaField("jahr", "STRING"),
        bigquery.SchemaField("monat", "STRING"),
        bigquery.SchemaField("monat_num", "INT64"),
        bigquery.SchemaField("name_spieler", "STRING"),
        bigquery.SchemaField("trainer", "STRING"),
        bigquery.SchemaField("standort", "STRING"),
        bigquery.SchemaField("position", "STRING"),
        bigquery.SchemaField("groesse", "INT64"),
        bigquery.SchemaField("nationalitaet", "STRING"),
        bigquery.SchemaField("alter", "STRING"),
        bigquery.SchemaField("paket", "STRING"),
        bigquery.SchemaField("vertrag", "STRING"),
        bigquery.SchemaField("kriterium", "STRING"),
        bigquery.SchemaField("bewertung", "FLOAT64"),
    ]
)
job_config_fussball_akademie.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

bq_client.load_table_from_dataframe(
    dataframe=df_melted,
    destination="web-scraping-371310.crawled_datasets.spielerbeurteilung",
    job_config=job_config_fussball_akademie
).result()

LoadJob<project=web-scraping-371310, location=EU, id=22316d16-8957-4afe-ba63-7095ef5151e0>

In [ ]:
# Select six columns from "df"
df_radar_view_test = df[[
    # ID columns
    "monat_jahr", "jahr", "monat", "monat_num", "name_spieler", "trainer", "standort", "bemerkungen_und_hinweise",
    
    # Numeric columns
    "jonglieren_mit_rueckdrall_links_und_rechts",
    "passspiel_innenseite_rechts", "passspiel_innenseite_links",
    "ballmitnahme_erster_kontakt_nach_vorne_rechte_innenseite",
    "ballmitnahme_erster_kontakt_nach_vorne_linke_innenseite",
    "ballmitnahme_erster_kontakt_90_grad_rechte_innenseite",
    "ballmitnahme_erster_kontakt_90_grad_linke_innenseite",
    "ballmitnahme_erster_kontakt_mit_offener_stellung_ca_90120_grad_rechte_innenseite",
    "ballmitnahme_erster_kontakt_mit_offener_stellung_ca_90120_grad_linke_innenseite",
    "ballmitnahme_erster_kontakt_90_grad_rechte_aussenseite",
    "ballmitnahme_erster_kontakt_90_grad_linke_aussenseite",
    "ballmitnahme_erster_kontakt_180_grad_rechte_aussenseite",
    "ballmitnahme_erster_kontakt_180_grad_linke_aussenseite",
    "jonglieren_mit_rueckdrall",
    "ballhaendling_innenseite_vollspann_oberschenkel_brust_liftenvolley",
    "enge_ballfuehrung_lr", "tempodribbling_lire", "koerperfinte_lire",
    "uebersteiger_lire", "torschuss_innenseite_lire",
    "torschuss_vollspann_lire", "1_gegen_1_offensiv", "1_gegen_1_defensiv",
    "antrittsschnelligkeit", "grundschnelligkeit",
    "laufbewegungsgeschicklichkeit",
    "aktionsschnelligkeit_bewegung_mit_ball",
    "spielintelligenz_kreativitaet", "durchsetzungsvermoegen_willenstaerke",
    "konzentrationsvermoegen", "selbstvertrauen", "kritikfaehigkeit",
    "disziplin", "ballfuehrung", "finten", "dribbling", "torschuss",
    "aktionsschnelligkeit", "durchsetzungsvermoegen", "willensstaerke"
]]

# Change all the data types of the numeric columns to float
df_radar_view_test.loc[:, "jonglieren_mit_rueckdrall_links_und_rechts":] = df_radar_view_test.loc[:, "jonglieren_mit_rueckdrall_links_und_rechts":].astype(float)

# Create the job configuration of df_booked_appts
job_config_radar_test = bigquery.LoadJobConfig(
    schema = [
        # ID columns
        bigquery.SchemaField("monat_jahr", "STRING"),
        bigquery.SchemaField("jahr", "STRING"),
        bigquery.SchemaField("monat", "STRING"),
        bigquery.SchemaField("monat_num", "INT64"),
        bigquery.SchemaField("name_spieler", "STRING"),
        bigquery.SchemaField("trainer", "STRING"),
        bigquery.SchemaField("standort", "STRING"),
        bigquery.SchemaField("bemerkungen_und_hinweise", "STRING"),

        # Numeric columns
        bigquery.SchemaField("jonglieren_mit_rueckdrall_links_und_rechts", "FLOAT64"),
        bigquery.SchemaField("passspiel_innenseite_rechts", "FLOAT64"),
        bigquery.SchemaField("passspiel_innenseite_links", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_nach_vorne_rechte_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_nach_vorne_linke_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_90_grad_rechte_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_90_grad_linke_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_mit_offener_stellung_ca_90120_grad_rechte_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_mit_offener_stellung_ca_90120_grad_linke_innenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_90_grad_rechte_aussenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_90_grad_linke_aussenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_180_grad_rechte_aussenseite", "FLOAT64"),
        bigquery.SchemaField("ballmitnahme_erster_kontakt_180_grad_linke_aussenseite", "FLOAT64"),
        bigquery.SchemaField("jonglieren_mit_rueckdrall", "FLOAT64"),
        bigquery.SchemaField("ballhaendling_innenseite_vollspann_oberschenkel_brust_liftenvolley", "FLOAT64"),
        bigquery.SchemaField("enge_ballfuehrung_lr", "FLOAT64"),
        bigquery.SchemaField("tempodribbling_lire", "FLOAT64"),
        bigquery.SchemaField("koerperfinte_lire", "FLOAT64"),
        bigquery.SchemaField("uebersteiger_lire", "FLOAT64"),
        bigquery.SchemaField("torschuss_innenseite_lire", "FLOAT64"),
        bigquery.SchemaField("torschuss_vollspann_lire", "FLOAT64"),
        bigquery.SchemaField("1_gegen_1_offensiv", "FLOAT64"),
        bigquery.SchemaField("1_gegen_1_defensiv", "FLOAT64"),
        bigquery.SchemaField("antrittsschnelligkeit", "FLOAT64"),
        bigquery.SchemaField("grundschnelligkeit", "FLOAT64"),
        bigquery.SchemaField("laufbewegungsgeschicklichkeit", "FLOAT64"),
        bigquery.SchemaField("aktionsschnelligkeit_bewegung_mit_ball", "FLOAT64"),
        bigquery.SchemaField("spielintelligenz_kreativitaet", "FLOAT64"),
        bigquery.SchemaField("durchsetzungsvermoegen_willenstaerke", "FLOAT64"),
        bigquery.SchemaField("konzentrationsvermoegen", "FLOAT64"),
        bigquery.SchemaField("selbstvertrauen", "FLOAT64"),
        bigquery.SchemaField("kritikfaehigkeit", "FLOAT64"),
        bigquery.SchemaField("disziplin", "FLOAT64"),
        bigquery.SchemaField("ballfuehrung", "FLOAT64"),
        bigquery.SchemaField("finten", "FLOAT64"),
        bigquery.SchemaField("dribbling", "FLOAT64"),
        bigquery.SchemaField("torschuss", "FLOAT64"),
        bigquery.SchemaField("aktionsschnelligkeit", "FLOAT64"),
        bigquery.SchemaField("durchsetzungsvermoegen", "FLOAT64"),
        bigquery.SchemaField("willensstaerke", "FLOAT64"),

    ]
)
job_config_radar_test.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

bq_client.load_table_from_dataframe(
    dataframe=df_radar_view_test,
    destination="web-scraping-371310.crawled_datasets.spielerbeurteilung_radar",
    job_config=job_config_radar_test
).result()

In [45]:
# Upload df_schwerpuntke to BigQuery
job_config_schwerpunkte = bigquery.LoadJobConfig(
    schema = [
        bigquery.SchemaField("schwerpunkt", "STRING"),
        bigquery.SchemaField("schwerpunkt_ohne_prefix", "STRING"),
        bigquery.SchemaField("schwerpunkt_id", "STRING"),
        bigquery.SchemaField("source", "STRING")
    ]
)
job_config_schwerpunkte.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

bq_client.load_table_from_dataframe(
    dataframe=df_schwerpuntke,
    destination="web-scraping-371310.crawled_datasets.spielerbeurteilung_schwerpunkte",
    job_config=job_config_schwerpunkte
).result()

LoadJob<project=web-scraping-371310, location=EU, id=a79403da-8899-4cae-a0ba-674176150b32>